In [2]:
# openai key is loaded in .env file. load the environment variables
from os import getenv
from dotenv import load_dotenv
# loadenv
load_dotenv(".env")

False

### Create OpenAI model
Two model - OpenAI and ChatOpenAI
With OpenAI, the input and output are strings, while with ChatOpenAI, the input is a sequence of messages and the output is a message.

In [3]:
from langchain_openai import OpenAI
# crate llm using openai 
llm = OpenAI(openai_api_key=getenv("OPENAI_API_KEY"))

In [4]:
res = llm.invoke("how are you?")

print(type(res))
print(res)

<class 'str'>


I am an AI assistant and do not have emotions, so I am always doing well. Thank you for asking. How can I assist you today?


In [5]:
from langchain_openai import ChatOpenAI
# chatopenai model
chat_llm = ChatOpenAI(openai_api_key=getenv("OPENAI_API_KEY"))

chat_res = chat_llm.invoke("how are you?")

print(type(chat_res))
print("AI Message:= ", chat_res)
print("Content:= ", chat_res.content)


<class 'langchain_core.messages.ai.AIMessage'>
AI Message:=  content="I'm just a computer program, so I don't have feelings or emotions, but I'm here and ready to assist you with any questions or tasks you have. How can I help you today?" response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 11, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None} id='run-20525c03-c88f-465e-b710-cd418a7c4fce-0'
Content:=  I'm just a computer program, so I don't have feelings or emotions, but I'm here and ready to assist you with any questions or tasks you have. How can I help you today?


In [6]:
# chatopen ai llm accept multiple messages
from langchain_core.messages import HumanMessage, SystemMessage

# chat history
# chain of messages and chat history
messages = [
    HumanMessage("Hello, my name is raghav. when you answer my questions, start addressing my name followed by my name"),
    SystemMessage("Hello raghav, how can I help you?"),
    HumanMessage("do any know steps to book flight?"),
]

res = chat_llm.invoke(messages)

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res.content)

ChatOpenAI message type:=  <class 'langchain_core.messages.ai.AIMessage'>
Response:=  Hello raghav, to book a flight, you can follow these steps:

1. Visit a flight booking website or app.
2. Enter your departure city, destination, travel dates, and number of passengers.
3. Browse through the available flights and select the one that suits your preferences.
4. Enter passenger details and payment information.
5. Review your booking details and confirm your flight reservation.
6. Receive a confirmation email with your flight details.

Is there anything else you would like to know about booking flights?


Output parser - parse the AIMessage using different parser

In [7]:
from langchain_core.output_parsers import StrOutputParser

# LCEL
llm_chat_str_parser = chat_llm | StrOutputParser()

res = llm_chat_str_parser.invoke(messages)

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res)

ChatOpenAI message type:=  <class 'str'>
Response:=  Hello raghav, to book a flight, you can follow these steps:
1. Visit a flight booking website or app.
2. Enter your departure city, destination, dates of travel, and number of passengers.
3. Browse through available flight options and select the one that suits your preferences and budget.
4. Enter passenger details and payment information.
5. Review your booking details and confirm your flight reservation.
6. Receive a confirmation email with your flight itinerary.

Is there anything else you would like to know about booking a flight?


# Prompts
A prompt for a language model is a set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

# chat prompt template
template = '''You are system administractor who can understand the resource utilization metrics of a system like cpu utilization, memory utilization, storage utilization and network utilization.
Translate the metrics details in below json format: 
All the metrics values must be in exact percentage.
{{
    "cpu": "cpu utilization in percentage",
    "memory": "memory utilization in percentage",
    "storage": "storage utilization in percentage",
    "network": "network utilization in percentage"
}}
system utilization metrics in unstructured format:
{resource_utilization}
'''
prompt = ChatPromptTemplate.from_template(template=template, input_args=["resource_utilization"])

llm_prompt = prompt | chat_llm

res = llm_prompt.invoke({"resource_utilization": "cpu utilization is 20%, memory utilization is 30%, storage utilization is 40%, network utilization is 50%"})

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res.content)
print("Content Type:= ", type(res.content))


ChatOpenAI message type:=  <class 'langchain_core.messages.ai.AIMessage'>
Response:=  {
    "cpu": "20%",
    "memory": "30%",
    "storage": "40%",
    "network": "50%"
}
Content Type:=  <class 'str'>


In [9]:
# parse the output using jsonparser 
from langchain_core.output_parsers import JsonOutputParser

llm_prompt_json_parser = llm_prompt | JsonOutputParser()

res = llm_prompt_json_parser.invoke({"resource_utilization": "cpu utilization is 20%, memory utilization is 30%, storage utilization is 40%, network utilization is 50%"})

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res)
print("CPU:= ", res['cpu'])


ChatOpenAI message type:=  <class 'dict'>
Response:=  {'cpu': '20%', 'memory': '30%', 'storage': '40%', 'network': '50%'}
CPU:=  20%


Parse output to Python object

In [11]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class ResourceUtilization(BaseModel):
    cpu: float = Field(..., description="cpu utilization in percentage")
    memory: float = Field(..., description="memory utilization in percentage")
    storage: float = Field(..., description="storage utilization in percentage")
    network: float = Field(..., description="network utilization in percentage")
    
pydantic_parser = PydanticOutputParser(pydantic_object=ResourceUtilization)

print( pydantic_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"cpu": {"title": "Cpu", "description": "cpu utilization in percentage", "type": "number"}, "memory": {"title": "Memory", "description": "memory utilization in percentage", "type": "number"}, "storage": {"title": "Storage", "description": "storage utilization in percentage", "type": "number"}, "network": {"title": "Network", "description": "network utilization in percentage", "type": "number"}}, "required": ["cpu", "memory", "storage", "network"]}
```


LLM with Pydantic Parser integration

In [13]:
# chat prompt template
template = '''You are system administractor who can understand the resource utilization metrics of a system like cpu utilization, memory utilization, storage utilization and network utilization.
{format_instructions}
System utilization metrics in unstructured format:
{resource_utilization}
'''
prompt = ChatPromptTemplate.from_template(template=template, 
                                          input_args=[
                                              "resource_utilization",
                                              "format_instructions"
                                            ]
                                          )

llm_prompt_json_parser = prompt | chat_llm | pydantic_parser

res = llm_prompt_json_parser.invoke({
    "format_instructions": pydantic_parser.get_format_instructions(),
    "resource_utilization": "cpu utilization is 20%, memory = 30%, storage utilization is 40%, network = 50%"})

print("ChatOpenAI message type:= ", type(res))
print("Response:= ", res)
print("CPU:= ", res.cpu)

ChatOpenAI message type:=  <class '__main__.ResourceUtilization'>
Response:=  cpu=20.0 memory=30.0 storage=40.0 network=50.0
CPU:=  20.0
